# Setup

### install required libraries

In [1]:
!pip install datasets # to get training data
!pip install transformers # to train whisper model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.9 MB/s eta 0:00:00


### HuggingFace Hub login

for common voice dataset access

In [4]:
from huggingface_hub import notebook_login

notebook_login()

# Get Dataset

We'll get the hindi subset of the Fleurs dataset and the Common Voice 13 dataset (more "Hindi" data)

In [5]:
from datasets import load_dataset, DatasetDict

fleurs = DatasetDict()
fleurs["train"] = load_dataset("google/fleurs", "hi_in", split="train")
fleurs["valid"] = load_dataset("google/fleurs", "hi_in", split="validation")

common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_13_0", "hi", split="train")
common_voice["valid"] = load_dataset("mozilla-foundation/common_voice_13_0", "hi", split="validation")

print(fleurs)
print(common_voice)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3487it [00:00, 72350.29it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 706it [00:00, 70568.10it/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 2120
    })
    valid: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 239
    })
})
DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 4479
    })
    valid: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 2281
    })
})


removing additional metadata information which we don't need

In [6]:
fleurs = fleurs.remove_columns(['id', 'num_samples', 'path', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'])
common_voice = common_voice.remove_columns(['client_id', 'path', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'])
print(fleurs)
print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2120
    })
    valid: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 239
    })
})
DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 4479
    })
    valid: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2281
    })
})


combining both datasets

In [7]:
from datasets import concatenate_datasets, Audio

# rename 'sentence' column in common voice dataset to align with fleurs dataset
common_voice = common_voice.rename_column('sentence', 'transcription')

# downsample audio in common voice dataset to align sampling rate with fleurs dataset
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

# now we combine
ds = DatasetDict()
ds['train'] = concatenate_datasets([fleurs['train'], common_voice['train']])
ds['valid'] = concatenate_datasets([fleurs['valid'], common_voice['valid']])

ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 6599
    })
    valid: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2520
    })
})

# Load a WhisperProcessor

It combines both whisper feature extractor and tokenizer.

Feature Extractor to pre-processes the raw audio inputs by padding/truncating them to length of 30s and then converting them to log-Mel spectrograms.

Tokenizer to post-processes the model outputs (index of predicted text) to text format.

In [8]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe") # these arguments specifies the tokenizer to append the language token and the task token to the start of the sequence

# Prepare dataset for training

In [14]:
# dataset preparation function
def prepare_dataset(batch):

    audio = batch["audio"]

    # compute log-Mel input features from input audio array using the feature extractor
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids using the tokenizer
    batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
    return batch


In [18]:
# apply the dataset preparation function to all training examples
ds = ds.map(prepare_dataset, remove_columns=ds.column_names["train"])
ds

Map:   0%|          | 0/6599 [00:00<?, ? examples/s]

Map:   0%|          | 0/2520 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 6599
    })
    valid: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2520
    })
})